In [ ]:
from reference.micrograd.micrograd.engine import Value
from reference.micrograd.micrograd.nn import Neuron, Layer, MLP
import json

def encode_nn(obj):
    if isinstance(obj, Layer):
        return { "neurons": obj.neurons }
    elif isinstance(obj, Neuron):
        weights = [{ "value": { "Value": { "id": v.id, "data": v.data } }, "input_node_id": 0 } for v in obj.w]
        neuron = {
            "weights": weights,
            "bias": { "id": obj.b.id, "data": obj.b.data }
        }
        if obj.nonlin:
            neuron["nonlin"] = { "Expr": { "id": 0, "op": { "Relu": 0 } }}
        return neuron
    elif isinstance(obj, MLP):
        return {
            "layers": obj.layers,
        }

def encode_mlp(model, output_file_path="mlp_init.json"):
    with open(output_file_path, "w") as f:
        f.write(json.dumps(model, default=encode_nn, indent=2))

In [ ]:
# [[[w.data for w in n.w] for n in l.neurons] for l in model.layers]
def encode_value(n):
    if isinstance(n, Value):
        if n.op == '':
            return { "Value": { "id": n.id, "data": n.data } }
        op = ''
        operands = [v.id for v in n.prev]
        if len(n.prev) == 1:
            operands = n.prev[0].id
        if n.op == "+":
            op = "Add"
        elif n.op == "sum":
            op = 'Sum'
        elif n.op == "*":
            op = "Mul"
        elif n.op == "ReLU":
            op = "Relu"
        elif n.op.startswith("**"):
            operands.push(n.op[2:])
            op = "Pow"
        cur = { "Expr": { "id": n.id, "op": { op: operands }}}
        return cur

def encode_graph(output_node, output_file_path="graph_init.json"):
    visited = set()
    topo = []
    def build_topo(n):
        if not n in visited:
            visited.add(n)
            for child in n.prev:
                build_topo(child)
            topo.append(n)

    build_topo(output_node)

    with open(output_file_path, "w") as f:
        f.write(json.dumps(topo, default=encode_value, indent=1))
    